**Hybrid neural network model**

In [ ]:
# Install the 'jupyterthemes' package using pip
# This package allows you to customize the appearance of Jupyter Notebooks
!pip install jupyterthemes

**Import essential libraries and modules**

In [ ]:
import os  # Provides functions to interact with the operating system (e.g., file paths)
import torch  # PyTorch main library for building and training neural networks
import PIL  # Python Imaging Library (used for image processing)
import torchvision  # Library for image datasets, models, and transformations
import random  # For generating random numbers (useful for data shuffling, augmentation, etc.)
import matplotlib.pyplot as plt  # For plotting images, graphs, etc.
import numpy as np  # For numerical operations and handling arrays

# Importing optimization tools from PyTorch
import torch.optim as optim

# Import image transformation utilities from torchvision
import torchvision.transforms as transforms

from PIL import Image  # For opening and manipulating image files

# Import neural network modules
from torch import nn, optim  # nn = neural network layers; optim = optimization algorithms
from torch.nn import functional as F  # Functional API for activation functions, loss, etc.

# More transformation tools from torchvision (aliased as T)
from torchvision import transforms as T
from torchvision import transforms

# Tools for data loading and dataset splitting
from torch.utils.data import DataLoader, Dataset, random_split

# Hugging Face Transformers library for using SegFormer model (semantic segmentation)
from transformers import SegformerForSemanticSegmentation, SegformerFeatureExtractor

# Import theme settings for Jupyter Notebook
from jupyterthemes import jtplot
jtplot.style()  # Apply the Jupyter notebook plot style to match the selected theme

**Check if a GPU (CUDA) is available.**

In [ ]:
# If available, use the GPU; otherwise, fall back to the CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Print the selected device (either 'cuda' or 'cpu')
print(device)

**Import the module to access Google Drive in Google Colab**

In [ ]:
from google.colab import drive

# Mount Google Drive to access its files from the Colab environment
# This will prompt the user to authorize access
drive.mount('/content/images')

# Set the path to a specific folder in your Google Drive
# Replace 'MyDrive/' with the actual path where your data is stored if needed
PATH = '/content/images/MyDrive/'

**Input images and corresponding masks for the model**

In [ ]:
# Set the path to the directory containing input images for the model
image_directory = "/content/images/MyDrive/Colab Notebooks/aug_imghybC"

# Set the path to the directory containing corresponding segmentation masks
mask_directory = "/content/images/MyDrive/aug_maskhybC"

**TensorFlow/Keras components**

In [ ]:
# Import NumPy for numerical operations and array handling
import numpy as np

# Import TensorFlow (the main deep learning library being used)
import tensorflow as tf

# Import core Keras layers for building CNN models
from tensorflow.keras.layers import (
    Input,             # Entry point for a model
    Conv2D,            # 2D convolution layer
    MaxPooling2D,      # Max pooling layer for downsampling
    UpSampling2D,      # Upsampling layer (often used in decoders like U-Net)
    Concatenate,       # Combine tensors along a specified axis
    Add,               # Element-wise addition
    Multiply           # Element-wise multiplication
)

# Import layers for classification and global feature aggregation
from tensorflow.keras.layers import (
    GlobalAveragePooling2D,  # Averages the spatial dimensions
    GlobalMaxPooling2D,      # Takes the max across spatial dimensions
    Dense                    # Fully connected layer
)

# Import the functional API model class
from tensorflow.keras.models import Model

# Keras backend, useful for custom functions (like loss or metrics)
from tensorflow.keras import backend as K

# Optimizer for training (Adam is commonly used for its efficiency)
from tensorflow.keras.optimizers import Adam

# Import additional layers
from tensorflow.keras.layers import BatchNormalization, Concatenate, Dropout

# Import callbacks for training:
# - EarlyStopping: Stops training when validation performance stops improving
# - ReduceLROnPlateau: Reduces learning rate when a metric has stopped improving
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Import a pre-trained ResNet50 model (can be used as a backbone for feature extraction)
from tensorflow.keras.applications import ResNet50

# Matplotlib for visualizing images and training curves
import matplotlib.pyplot as plt

# OpenCV (cv2) for advanced image processing tasks (reading, resizing, augmenting, etc.)
import cv2


**Channel-wise attention**

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Reshape, Multiply

def channel_attention(x, ratio=16):
    # Channel Attention Mechanism using Squeeze-and-Excitation (SE) block

    channel = x.shape[-1]  # Get the number of channels in the input feature map

    # Step 1: Squeeze — Global Average Pooling
    # Reduces the spatial dimensions (H x W) to a single vector (C,)
    se = GlobalAveragePooling2D()(x)

    # Step 2: Excitation — Fully connected layers
    # First dense layer reduces the number of channels (bottleneck)
    se = Dense(channel // ratio, activation='relu')(se)

    # Second dense layer restores the original channel size and uses sigmoid activation
    se = Dense(channel, activation='sigmoid')(se)

    # Step 3: Reshape — Convert to (1, 1, C) so it can be multiplied with the input
    se = Reshape((1, 1, channel))(se)

    # Step 4: Scale — Multiply the input feature map by the attention weights
    return Multiply()([x, se])

**Spatial Attention mechanism**

In [ ]:
from tensorflow.keras.layers import Layer

class SpatialAttention(Layer):
    def __init__(self, **kwargs):
        super(SpatialAttention, self).__init__(**kwargs)
        # Convolution layer with 1 output channel and sigmoid activation
        # Kernel size 7x7 to capture spatial context
        self.conv = Conv2D(1, (7, 7), padding='same', activation='sigmoid')

    def call(self, x):
        # Compute average pooling along the channel axis (C)
        avg_pool = tf.reduce_mean(x, axis=-1, keepdims=True)

        # Compute max pooling along the channel axis (C)
        max_pool = tf.reduce_max(x, axis=-1, keepdims=True)

        # Concatenate both pooling outputs along the channel dimension
        concat = tf.concat([avg_pool, max_pool], axis=-1)

        # Apply 2D convolution to generate spatial attention map
        attention_map = self.conv(concat)

        # Multiply the attention map with the input feature map
        return Multiply()([x, attention_map])

**Feature refinement block**

In [ ]:
def conv_block(x, filters, kernel_size=3, padding='same', activation='relu'):
    # Apply two Conv2D layers with the specified number of filters and activation
    x = Conv2D(filters, kernel_size, padding=padding, activation=activation)(x)
    x = Conv2D(filters, kernel_size, padding=padding, activation=activation)(x)

    # Apply Channel Attention (Squeeze-and-Excitation)
    x = channel_attention(x)

    # Apply Spatial Attention
    x = SpatialAttention()(x)

    return x

**Encoder block**

In [ ]:
def encoder_block(x, filters):
    # Apply a convolutional block with attention (Conv + Channel + Spatial Attention)
    x = conv_block(x, filters)

    # Downsample the feature map using MaxPooling
    p = MaxPooling2D(pool_size=(2, 2))(x)

    # Return both the feature map (x) and the pooled output (p)
    return x, p

**Decoder block**

In [ ]:
def decoder_block(x, skip, filters):
    # Step 1: Upsample using a transposed convolution
    x = tf.keras.layers.Conv2DTranspose(filters, (3, 3), strides=(2, 2), padding='same')(x)

    # Step 2: Compute required padding to match dimensions with the skip connection
    pad_height = skip.shape[1] - x.shape[1]
    pad_width = skip.shape[2] - x.shape[2]

    if pad_height > 0 or pad_width > 0:
        # Apply zero-padding to align spatial dimensions with the skip connection
        x = tf.keras.layers.ZeroPadding2D(padding=((pad_height // 2, pad_height - pad_height // 2),
                                                   (pad_width // 2, pad_width - pad_width // 2)))(x)

    # Step 3: Concatenate the upsampled tensor with the corresponding encoder feature map (skip connection)
    x = tf.keras.layers.Concatenate()([x, skip])

    # Step 4: Apply convolutional block with attention mechanisms
    x = conv_block(x, filters)

    return x

**Atrous Spatial Pyramid Pooling (ASPP) for capturing multi-scale context**

In [ ]:
def aspp_block(x):

    # 1x1 convolution (no dilation) – captures local features
    aspp1 = Conv2D(256, 1, padding='same', activation='relu')(x)

    # 3x3 convolutions with increasing dilation rates to capture multi-scale features
    aspp2 = Conv2D(256, 3, padding='same', dilation_rate=6, activation='relu')(x)
    aspp3 = Conv2D(256, 3, padding='same', dilation_rate=12, activation='relu')(x)
    aspp4 = Conv2D(256, 3, padding='same', dilation_rate=18, activation='relu')(x)

    # Additional 1x1 convolution (optional—can be considered a redundant path or auxiliary signal)
    aspp5 = Conv2D(256, 1, padding='same', activation='relu')(x)

    # Concatenate all ASPP branches along the channel axis
    x = Concatenate()([aspp1, aspp2, aspp3, aspp4, aspp5])

    # Reduce channel depth after concatenation with another 1x1 conv
    x = Conv2D(256, 1, padding='same', activation='relu')(x)

    return x

**Deep encoder-decoder CNN for image segmentation**, with:

(a) Attention blocks (channel + spatial)

(b) ASPP (Atrous Spatial Pyramid Pooling)

(c) LSTM layer in the bottleneck to model long-range dependencies

(d) level depth (deeper than standard U-Net)

(e) Dropout for regularization

In [ ]:
def build_deeper_model_with_lstm(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS, dropout_rate=0.5):

    # Define the input shape of the model based on image dimensions and channels.
    input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    inputs = Input(shape=input_shape)

    # Encoder Path (Feature Extraction with Downsampling)
    # Apply 6 encoder blocks with increasing filter sizes (from 64 to 2048).

    # Each block uses:
    # 2 Conv2D layers
    # Channel and spatial attention
    # MaxPooling
    # Dropout for regularization

    e1, p1 = encoder_block(inputs, 64)
    e1 = Dropout(dropout_rate)(e1)
    e2, p2 = encoder_block(p1, 128)
    e2 = Dropout(dropout_rate)(e2)
    e3, p3 = encoder_block(p2, 256)
    e3 = Dropout(dropout_rate)(e3)
    e4, p4 = encoder_block(p3, 512)
    e4 = Dropout(dropout_rate)(e4)
    e5, p5 = encoder_block(p4, 1024)
    e5 = Dropout(dropout_rate)(e5)
    e6, p6 = encoder_block(p5, 2048)
    e6 = Dropout(dropout_rate)(e6)

    # Bottleneck + ASPP

    # The deepest layer (b1) is processed by a convolutional block.
    # Then passed through the ASPP block to extract multi-scale contextual features.
    b1 = conv_block(p6, 4096)
    b1 = Dropout(dropout_rate)(b1)
    b2 = aspp_block(b1)

    # LSTM Integration (Temporal/Sequential Context in Spatial Features)

    # ASPP output is reshaped into a sequence format for LSTM input: shape → (timesteps, features).
    # LSTM captures spatial dependencies along flattened feature sequences.
    # The output is reshaped back into spatial dimensions to continue decoding.

    lstm_input = tf.keras.layers.Reshape((b2.shape[1] * b2.shape[2], b2.shape[3]))(b2)
    lstm_out = tf.keras.layers.LSTM(512, return_sequences=True)(lstm_input)
    lstm_out = Dropout(dropout_rate)(lstm_out)
    lstm_out = Reshape((IMG_HEIGHT // 64, IMG_WIDTH // 64, 512))(lstm_out)

    # Decoder Path (Upsampling + Skip Connections)

    # Each decoder block:
    # Upsamples using transposed convolution
    # Aligns and concatenates with the corresponding encoder output (skip connection)
    # Refines with conv + attention block
    # Dropout is applied after each decoder stage

    d1 = decoder_block(lstm_out, e6, 1024)
    d1 = Dropout(dropout_rate)(d1)
    d2 = decoder_block(d1, e5, 512)
    d2 = Dropout(dropout_rate)(d2)
    d3 = decoder_block(d2, e4, 256)
    d3 = Dropout(dropout_rate)(d3)
    d4 = decoder_block(d3, e3, 128)
    d4 = Dropout(dropout_rate)(d4)
    d5 = decoder_block(d4, e2, 64)
    d5 = Dropout(dropout_rate)(d5)
    d6 = decoder_block(d5, e1, 32)
    d6 = Dropout(dropout_rate)(d6)

    # Output Layer
    outputs = Conv2D(1, 1, activation='sigmoid')(d6)

    # Model Compilation
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

**Importing libraries and modules**

In [ ]:
# Import the normalize function from Keras utilities for data normalization
from keras.utils import normalize

# Import the os module to interact with the operating system (e.g., file paths)
import os

# Import OpenCV library for image processing tasks
import cv2

# Import PIL (Pillow) library for image manipulation
from PIL import Image

# Import NumPy for numerical operations and array handling
import numpy as np

# Import pyplot module from matplotlib for plotting and visualization
from matplotlib import pyplot as plt

# Import ThreadPoolExecutor for concurrent execution of code using threads
from concurrent.futures import ThreadPoolExecutor

**Target size**

In [ ]:
# Define the target size (width and height) for images to be processed
SIZE = 224

# Initialize an empty list to store image data
image_dataset = []

# Initialize an empty list to store corresponding mask data
mask_dataset = []

**Lists of filenames from the image and mask directories**

In [ ]:
# Get sorted lists of filenames from the image and mask directories
images = sorted(os.listdir(image_directory))
masks = sorted(os.listdir(mask_directory))

# Ensure the number of images matches the number of masks
assert len(images) == len(masks), "The number of images and masks does not match"

# Define a function to load and preprocess an image-mask pair
def process_image_mask(image_name, mask_name):
    # Check if both files have the '.tif' extension
    if image_name.split('.')[1] == 'tif' and mask_name.split('.')[1] == 'tif':
        # Construct full file paths for image and mask
        image_path = os.path.join(image_directory, image_name)
        mask_path = os.path.join(mask_directory, mask_name)

        # Load the image in grayscale mode
        image = cv2.imread(image_path, 0)
        if image is None:
            print(f"Failed to load image: {image_path}")
            return None, None

        # Load the mask in grayscale mode
        mask = cv2.imread(mask_path, 0)
        if mask is None:
            print(f"Failed to load mask: {mask_path}")
            return None, None

        # Resize both image and mask to the target size
        image = cv2.resize(image, (SIZE, SIZE))
        mask = cv2.resize(mask, (SIZE, SIZE))

        # Convert images to numpy arrays and return them
        return np.array(image), np.array(mask)

    # Return None if file extensions are not '.tif'
    return None, None

# Initialize empty lists to store processed images and masks
image_dataset = []
mask_dataset = []

# Use ThreadPoolExecutor to process image-mask pairs concurrently
with ThreadPoolExecutor() as executor:
    # Map the processing function to each pair of filenames
    results = list(executor.map(lambda pair: process_image_mask(*pair), zip(images, masks)))

# Append successfully processed image-mask pairs to the datasets
for image, mask in results:
    if image is not None and mask is not None:
        image_dataset.append(image)
        mask_dataset.append(mask)

# Indicate that processing is complete
print("Processing completed")

In [ ]:
# Convert the list of images to a NumPy array, normalize pixel values along axis 1,
# and expand dimensions to add a channel dimension at the end (grayscale channel)
image_dataset = np.expand_dims(normalize(np.array(image_dataset), axis=1), 3)

# Convert the list of masks to a NumPy array, expand dimensions to add a channel dimension,
# and normalize mask pixel values to the range [0, 1] by dividing by 255
mask_dataset = np.expand_dims(np.array(mask_dataset), 3) / 255.

In [ ]:
# Import train_test_split function from scikit-learn to split datasets
from sklearn.model_selection import train_test_split

# Split the image and mask datasets into training and testing sets
# 20% of the data is reserved for testing; random_state ensures reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset, mask_dataset, test_size=0.2, random_state=0
)

# Further split the training set into a smaller training subset and a quick test subset
# This can be used for quicker experiments or validation during development
X_train_quick_test, X_test_quick_test, y_train_quick_test, y_test_quick_test = train_test_split(
    X_train, y_train, test_size=0.2, random_state=0
)

In [ ]:
# Import required modules for randomness and numerical operations
import random
import numpy as np

# Randomly select an index from the quick test training set
image_number = random.randint(0, len(X_train_quick_test))

# Set the figure size for plotting
plt.figure(figsize=(12, 6))

# Display the input image in the first subplot
plt.subplot(121)
plt.imshow(np.reshape(X_train_quick_test[image_number], (224, 224)), cmap='gray')
plt.title("Input Image")

# Display the corresponding mask in the second subplot
plt.subplot(122)
plt.imshow(np.reshape(y_train_quick_test[image_number], (224, 224)), cmap='gray')
plt.title("Mask")

# Show the plots
plt.show()

**ImageDataGenerator for performing data augmentation**

In [ ]:
# Import ImageDataGenerator for performing data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the data augmentation configuration
datagen = ImageDataGenerator(
    rotation_range=20,           # Randomly rotate images in the range (degrees)
    width_shift_range=0.2,       # Randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,      # Randomly shift images vertically (fraction of total height)
    shear_range=0.2,             # Shear angle in counter-clockwise direction (in degrees)
    zoom_range=0.2,              # Random zoom within the specified range
    horizontal_flip=True,        # Randomly flip images horizontally
    fill_mode='nearest'          # Strategy for filling in newly created pixels after rotation or shifting
)

**Configured hybrid model**

In [ ]:
# Extract the input image dimensions and number of channels from the dataset
IMG_HEIGHT = image_dataset.shape[1]
IMG_WIDTH  = image_dataset.shape[2]
IMG_CHANNELS = image_dataset.shape[3]

# Define a function that returns a model configured with the appropriate input shape
# Assumes that 'build_deeper_model_with_lstm' is a custom model-building function defined elsewhere
def get_jacard_model():
    return build_deeper_model_with_lstm(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

# Instantiate the model using the defined function
model_jacard = get_jacard_model()

**Callbacks**

In [ ]:
# Import training callbacks from Keras
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define early stopping to prevent overfitting
# Training will stop if the validation loss does not improve for 3 consecutive epochs
# The model will restore the weights from the epoch with the best validation loss
early_stopping = EarlyStopping(
    monitor='val_loss',           # Metric to monitor
    patience=3,                   # Number of epochs with no improvement before stopping
    restore_best_weights=True     # Restore model weights from the epoch with the best value
)

# Define learning rate reduction strategy
# Reduces the learning rate by a factor of 0.5 if the validation loss does not improve for 2 epochs
# Ensures that the learning rate never drops below 1e-6
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',           # Metric to monitor
    factor=0.5,                   # Factor by which the learning rate will be reduced
    patience=2,                   # Number of epochs with no improvement before reducing LR
    min_lr=1e-6                   # Minimum learning rate
)

In [ ]:
# Re-instantiate the model by calling the model-building function
# This is useful if you want to reset the model (e.g., before retraining)
model_jacard = get_jacard_model()

**Pre-trained weights**

In [ ]:
# Load pre-trained weights into the model from a .h5 file
# This allows you to resume training or perform evaluation/inference using saved model parameters
model_jacard.load_weights('/content/images/MyDrive/CNN-hybrid_weightsCont5.h5')

**Training with Data generator**

In [ ]:
# Train the model using augmented data from the training subset
history_jacard = model_jacard.fit(
    datagen.flow(X_train_quick_test, y_train_quick_test, batch_size=16),  # Data generator with augmentation
    verbose=1,                                                            # Print detailed training progress
    epochs=20,                                                            # Train for 20 epochs
    validation_data=(X_test, y_test),                                     # Use full test set for validation
    callbacks=[early_stopping, reduce_lr],                                # Apply early stopping and learning rate reduction
    shuffle=False                                                         # Do not shuffle data between epochs
)

**Training without Data generator**

In [ ]:
# Train the model directly on the training subset (without data augmentation)
history_jacard = model_jacard.fit(
    X_train_quick_test,               # Training images
    y_train_quick_test,              # Corresponding training masks
    batch_size=16,                   # Number of samples per gradient update
    verbose=1,                       # Display detailed logs during training
    epochs=10,                       # Train the model for up to 50 epochs
    validation_data=(X_test, y_test),# Evaluate the model on the full test set after each epoch
    callbacks=[early_stopping, reduce_lr],  # Apply early stopping and learning rate reduction
    shuffle=False                    # Do not shuffle the data (important if order matters)
)

**Save de entire model**

In [ ]:
# Save the entire model (architecture + weights + optimizer state) to an HDF5 file
# This allows the model to be reloaded later using keras.models.load_model()
model_jacard.save('/content/images/MyDrive/CNN-hybrid_weightsCont5.h5')

**Model's performance**

In [ ]:
# Evaluate the model's performance on the test set
# Returns loss and any additional metrics (e.g., accuracy, if configured)
_, accuracy = model_jacard.evaluate(X_test, y_test)

# Print the accuracy as a percentage
print("Accuracy of Jacard Model is = ", (accuracy * 100.0), "%")

In [ ]:
# Extract training and validation loss values from the training history
loss = history_jacard.history['loss']
val_loss = history_jacard.history['val_loss']

# Create a range of epoch numbers for the x-axis
epochs = range(1, len(loss) + 1)

# Plot training loss in yellow
plt.plot(epochs, loss, 'y', label='Training loss')

# Plot validation loss in red
plt.plot(epochs, val_loss, 'r', label='Validation loss')

# Set chart title and axis labels
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Display the legend and the plot
plt.legend()
plt.show()

**IoU (Jaccard Index)**

In [ ]:
# Assign the model to a variable for clarity
model = model_jacard

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Apply a threshold to convert predicted probabilities to binary mask (True/False)
y_pred_thresholded = y_pred > 0.1

# Calculate the intersection (logical AND) between ground truth and prediction masks
intersection = np.logical_and(y_test, y_pred_thresholded)

# Calculate the union (logical OR) between ground truth and prediction masks
union = np.logical_or(y_test, y_pred_thresholded)

# Compute the Intersection over Union (IoU) score
iou_score = np.sum(intersection) / np.sum(union)

# Print the IoU score
print("IoU score is: ", iou_score)

**Dice coefficient**

In [ ]:
# Import NumPy for numerical operations
import numpy as np

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Apply a threshold to convert predicted probabilities to binary masks
y_pred_thresholded = y_pred > 0.1

# Calculate the intersection between ground truth and predicted masks
intersection = np.logical_and(y_test, y_pred_thresholded)
intersection_count = np.sum(intersection)

# Calculate the total number of positive pixels in ground truth and prediction
y_test_count = np.sum(y_test)
y_pred_count = np.sum(y_pred_thresholded)

# Compute the Dice coefficient (F1 score for segmentation)
dice_coefficient = (2 * intersection_count) / (y_test_count + y_pred_count)

# Print the Dice coefficient
print("Dice coefficient is: ", dice_coefficient)

**Jaccard coefficient**

In [ ]:
# Generate predictions on the test set
y_pred = model.predict(X_test)

# Apply threshold to convert predicted probabilities into binary masks
y_pred_thresholded = y_pred > 0.1

# Compute intersection: pixels correctly predicted as foreground
intersection = np.logical_and(y_test, y_pred_thresholded)

# Compute union: all pixels that are foreground in either ground truth or prediction
union = np.logical_or(y_test, y_pred_thresholded)

# Calculate the Jaccard coefficient (Intersection over Union)
jaccard_score = np.sum(intersection) / np.sum(union)

# Print the Jaccard coefficient
print("Jaccard coefficient is: ", jaccard_score)

**Precision, Recall, F1-score**

In [ ]:
# Import precision, recall, and F1-score metrics from scikit-learn
from sklearn.metrics import precision_score, recall_score, f1_score

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Threshold the predicted probabilities to create binary masks
y_pred_thresholded = y_pred > 0.1  # Use 0.1 as the classification threshold

# Convert ground truth masks to binary format if they aren't already
y_test_binary = y_test > 0.1

# Flatten the masks to 1D arrays for metric calculation
y_true_flat = y_test_binary.flatten()
y_pred_flat = y_pred_thresholded.flatten()

# Calculate precision: TP / (TP + FP)
precision = precision_score(y_true_flat, y_pred_flat)

# Calculate recall: TP / (TP + FN)
recall = recall_score(y_true_flat, y_pred_flat)

# Calculate F1-score: harmonic mean of precision and recall
f1 = f1_score(y_true_flat, y_pred_flat)

# Display the computed metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

**Mean Coverage (mCov) score**

In [ ]:
# Import NumPy for array operations
import numpy as np

# Generate predictions on the test set
y_pred = model.predict(X_test)

# Apply threshold to convert predicted probabilities to binary masks
y_pred_thresholded = y_pred > 0.1

# Convert ground truth masks to binary format
y_test_binary = y_test > 0.1

# Ensure predictions and ground truth masks have the same shape
assert y_pred_thresholded.shape == y_test_binary.shape

# Initialize list to store coverage scores for each image
num_images = y_test.shape[0]
coverage_scores = []

# Loop over each image in the test set
for i in range(num_images):
    pred = y_pred_thresholded[i].astype(bool).flatten()  # Flatten prediction mask
    gt = y_test_binary[i].astype(bool).flatten()         # Flatten ground truth mask

    # Compute intersection between prediction and ground truth
    intersection = np.logical_and(pred, gt)

    # Compute the area of the ground truth mask
    gt_area = np.sum(gt)

    # Skip images with empty ground truth masks (avoid division by zero)
    if gt_area == 0:
        continue

    # Calculate coverage as intersection over ground truth area
    coverage = np.sum(intersection) / gt_area
    coverage_scores.append(coverage)

# Compute mean coverage score across all valid images
if coverage_scores:
    mCov = np.mean(coverage_scores)
else:
    mCov = 0.0

# Print the mean coverage score
print("Mean Coverage (mCov) score:", mCov)

**Predictions**

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt

# Define a function to visualize predictions vs ground truth
def visualize_predictions(model, X_test, y_test, threshold=0.1, num_images=5):

    # Generate predictions from the model
    preds = model.predict(X_test)

    # Convert predicted probabilities to binary masks using the given threshold
    preds_binary = (preds > threshold).astype(np.uint8)

    # Ensure we do not exceed the number of available test images
    num_images = min(num_images, len(X_test))

    # Randomly select a subset of indices to visualize
    random_indices = np.random.choice(len(X_test), num_images, replace=False)

    # Iterate over selected images and display input, ground truth, and prediction
    for i in random_indices:
        plt.figure(figsize=(15, 5))

        # Show the original input image
        plt.subplot(1, 3, 1)
        plt.title('Original Image')
        plt.imshow(X_test[i].reshape(224, 224), cmap='gray')  # Assuming grayscale images
        plt.axis('off')

        # Show the ground truth mask
        plt.subplot(1, 3, 2)
        plt.title('Ground Truth Mask')
        plt.imshow(y_test[i].reshape(224, 224), cmap='gray')  # Assuming grayscale masks
        plt.axis('off')

        # Show the predicted mask
        plt.subplot(1, 3, 3)
        plt.title(f'Predicted Mask (Threshold={threshold})')
        plt.imshow(preds_binary[i].reshape(224, 224), cmap='viridis')  # Color map can be adjusted
        plt.axis('off')

        # Display the plots
        plt.show()

# Call the function to visualize predictions on 5 random test samples
visualize_predictions(model, X_test, y_test, threshold=0.1, num_images=5)

**Predictions of masks with edges**

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import sobel

# Function to extract and highlight edges from a prediction mask using Sobel filter
def highlight_edges(image, threshold=0.1):
    # Compute gradients along x and y axes
    sobel_x = sobel(image, axis=0)
    sobel_y = sobel(image, axis=1)

    # Combine gradients to get edge magnitude
    edges = np.hypot(sobel_x, sobel_y)

    # Normalize edge values to range [0, 255]
    edges = np.uint8(255 * edges / np.max(edges))

    # Apply threshold to filter out weak edges
    edges[edges < threshold * 255] = 0

    return edges

# Function to visualize a few test predictions with edge enhancement
def visualize_predictions(model, X_test, y_test, threshold=0.1, num_images=5):
    # Generate predictions from the model
    preds = model.predict(X_test)

    # Randomly select a subset of test images
    random_indices = np.random.choice(len(X_test), num_images, replace=False)

    # Loop through selected samples and visualize them
    for i in random_indices:
        plt.figure(figsize=(18, 6))

        # Original input image
        plt.subplot(1, 3, 1)
        plt.title('Original Image', fontsize=16)
        original_image = X_test[i].reshape(224, 224)
        plt.imshow(original_image, cmap='gray')
        plt.axis('off')
        plt.gca().add_patch(plt.Rectangle((0, 0), 224, 224, linewidth=2, edgecolor='black', facecolor='none'))

        # Ground truth segmentation mask
        plt.subplot(1, 3, 2)
        plt.title('Ground Truth Mask', fontsize=16)
        ground_truth = y_test[i].reshape(224, 224)
        plt.imshow(ground_truth, cmap='gray')
        plt.axis('off')
        plt.gca().add_patch(plt.Rectangle((0, 0), 224, 224, linewidth=2, edgecolor='black', facecolor='none'))

        # Predicted mask with edge highlights
        plt.subplot(1, 3, 3)
        plt.title('Predicted Mask with Edges', fontsize=16)
        predicted_mask = preds[i].reshape(224, 224)
        predicted_with_edges = highlight_edges(predicted_mask, threshold)
        plt.imshow(predicted_with_edges, cmap='inferno')
        plt.axis('off')
        plt.gca().add_patch(plt.Rectangle((0, 0), 224, 224, linewidth=2, edgecolor='black', facecolor='none'))

        # Improve layout and display
        plt.tight_layout()
        plt.show()

# Call the function to visualize 5 test predictions with edge detection
visualize_predictions(model, X_test, y_test, threshold=0.1, num_images=5)

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import sobel
import random

# Function to extract edges using the Sobel operator and apply thresholding
def highlight_edges(image, threshold=0.1):
    # Compute gradients along the x and y axes
    sobel_x = sobel(image, axis=0)
    sobel_y = sobel(image, axis=1)

    # Combine the gradients to get edge magnitude
    edges = np.hypot(sobel_x, sobel_y)

    # Normalize edges to 0–255 scale
    edges = np.uint8(255 * edges / np.max(edges))

    # Apply a threshold to remove weak edges
    edges[edges < threshold * 255] = 0

    return edges

# Function to visualize original image, ground truth, and predicted mask with edge overlay
def visualize_predictions(model, X_test, y_test, threshold=0.1, num_images=5):
    # Generate predictions from the model
    preds = model.predict(X_test)

    # Randomly select image indices
    random_indices = random.sample(range(len(X_test)), num_images)

    # Iterate through selected samples
    for i in random_indices:
        plt.figure(figsize=(18, 6))

        # --- Panel 1: Original Image ---
        plt.subplot(1, 3, 1)
        plt.title('Original Image', fontsize=16)
        original_image = X_test[i].reshape(224, 224)
        plt.imshow(original_image, cmap='gray')
        plt.axis('off')
        plt.gca().add_patch(plt.Rectangle((0, 0), 224, 224, linewidth=2, edgecolor='black', facecolor='none'))

        # --- Panel 2: Ground Truth Mask ---
        plt.subplot(1, 3, 2)
        plt.title('Ground Truth Mask', fontsize=16)
        ground_truth = y_test[i].reshape(224, 224)
        plt.imshow(ground_truth, cmap='gray')
        plt.axis('off')
        plt.gca().add_patch(plt.Rectangle((0, 0), 224, 224, linewidth=2, edgecolor='black', facecolor='none'))

        # --- Panel 3: Overlay Predicted Edges on Original Image ---
        plt.subplot(1, 3, 3)
        plt.title('Predicted Mask with Edges', fontsize=16)
        predicted_mask = preds[i].reshape(224, 224)

        # Highlight edges from the predicted mask
        predicted_with_edges = highlight_edges(predicted_mask, threshold)

        # Overlay the edges on the original image
        plt.imshow(original_image, cmap='gray')  # Base grayscale image
        plt.imshow(predicted_with_edges, cmap='jet', alpha=0.65)  # Edge heatmap overlay
        plt.axis('off')
        plt.gca().add_patch(plt.Rectangle((0, 0), 224, 224, linewidth=2, edgecolor='black', facecolor='none'))

        # Improve layout
        plt.tight_layout()
        plt.show()

# Call the visualization function
visualize_predictions(model, X_test, y_test, threshold=0.1, num_images=5)

**Error map**

In [ ]:

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

# Generate predictions from the model
preds = model.predict(X_test, batch_size=32)

# Apply threshold to convert prediction probabilities into binary masks
y_pred_thresholded = preds > 0.1

# Select a random image index from the test set
test_img_number = random.randint(0, len(X_test) - 1)

# Extract the ground truth and predicted masks for the selected image
true_mask = y_test[test_img_number, :, :, 0]
pred_mask = y_pred_thresholded[test_img_number, :, :, 0]

# Calculate the absolute error map (difference between ground truth and prediction)
error_map = np.abs(true_mask - pred_mask)

# --- Plot original image, ground truth, and predicted mask ---
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.title('Original Image')
plt.imshow(X_test[test_img_number, :, :, 0], cmap='gray')
plt.axis(False)  # Remove axes
plt.grid(False)

plt.subplot(1, 3, 2)
plt.title('Ground Truth Mask')
plt.imshow(true_mask, cmap='gray')
plt.axis(False)
plt.grid(False)

plt.subplot(1, 3, 3)
plt.title('Predicted Mask')
plt.imshow(pred_mask, cmap='gray')
plt.axis(False)
plt.grid(False)

# --- Plot the error map ---
plt.figure(figsize=(10, 5))
plt.title('Error Map')
plt.imshow(error_map, cmap='hot')  # 'hot' colormap highlights error intensities
plt.colorbar()
plt.axis(False)
plt.grid(False)
plt.show()

**Morphology**

In [ ]:
def apply_morphology(prediction, operation='closing', kernel_size=(5, 5)):
    """
    Apply a specified morphological operation to a batch of binary prediction masks.

    Parameters:
        prediction (numpy.ndarray): Batch of predicted binary masks with shape (N, H, W, 1).
        operation (str): Morphological operation to apply. One of:
                         'closing', 'opening', 'dilation', 'erosion'.
        kernel_size (tuple): Size of the structuring element (default is (5, 5)).

    Returns:
        numpy.ndarray: Batch of morphologically processed masks with the same shape as input.
    """
    # Define the structuring element (kernel) using a rectangular shape
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)

    morphed_predictions = []

    # Process each image individually
    for i in range(prediction.shape[0]):
        img = prediction[i, :, :, 0]  # Extract single mask (H, W)

        # Apply the selected morphological operation
        if operation == 'closing':
            morphed_img = cv2.morphologyEx(img.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
        elif operation == 'opening':
            morphed_img = cv2.morphologyEx(img.astype(np.uint8), cv2.MORPH_OPEN, kernel)
        elif operation == 'dilation':
            morphed_img = cv2.dilate(img.astype(np.uint8), kernel)
        elif operation == 'erosion':
            morphed_img = cv2.erode(img.astype(np.uint8), kernel)
        else:
            raise ValueError("Invalid operation. Use 'closing', 'opening', 'dilation', or 'erosion'.")

        # Add channel dimension back (H, W, 1)
        morphed_predictions.append(morphed_img[..., np.newaxis])

    # Stack all processed masks into a batch (N, H, W, 1)
    return np.stack(morphed_predictions, axis=0)

In [ ]:
# Predict masks for the test set using the trained model
y_pred = model_jacard.predict(X_test)

# Threshold the predictions to get binary masks
y_pred_thresholded = (y_pred > 0.1).astype(np.uint8)

# Apply morphological closing operation to clean up the predicted masks
y_pred_morphed = apply_morphology(y_pred_thresholded, operation='closing')

# Calculate the Intersection over Union (IoU) score between the ground truth and morphologically processed predictions
intersection = np.logical_and(y_test, y_pred_morphed)
union = np.logical_or(y_test, y_pred_morphed)
iou_score = np.sum(intersection) / np.sum(union)

# Print the IoU score after morphology
print("IoU score after morphology is: ", iou_score)

In [ ]:
def visualize_results(original_image, ground_truth, prediction, morphed_prediction):
    """
    Visualize the original image, ground truth mask, predicted mask (before and after morphology).

    Parameters:
        original_image (numpy.ndarray): The original input image (H, W).
        ground_truth (numpy.ndarray): The ground truth mask (H, W).
        prediction (numpy.ndarray): The predicted binary mask without morphology (H, W).
        morphed_prediction (numpy.ndarray): Batch of morphologically processed masks (N, H, W, 1).
        test_img_number (int): Index of the image in the morphed_prediction batch to visualize.
    """
    plt.figure(figsize=(12, 8))

    plt.subplot(221)
    plt.title('Original Image')
    plt.imshow(original_image, cmap='gray')
    plt.axis('off')

    plt.subplot(222)
    plt.title('Ground Truth Mask')
    plt.imshow(ground_truth, cmap='gray')
    plt.axis('off')

    plt.subplot(223)
    plt.title('Prediction without Morphology')
    plt.imshow(prediction, cmap='gray')
    plt.axis('off')

    plt.subplot(224)
    plt.title('Prediction with Morphology')
    plt.imshow(morphed_prediction[test_img_number, :, :, 0], cmap='gray')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# Predict segmentation masks on the test set using the trained model
y_pred = model_jacard.predict(X_test)

# Apply a threshold to convert predicted probabilities into binary masks
# Pixels with a prediction probability greater than 0.1 are set to 1, else 0
y_pred_thresholded = (y_pred > 0.1).astype(np.uint8)

# Apply morphological closing operation to the thresholded predictions
# This helps to remove small holes and smooth the mask edges
y_pred_morphed = apply_morphology(y_pred_thresholded, operation='closing')

# Randomly select an image index from the test set for visualization
test_img_number = random.randint(0, len(X_test))

# Retrieve the original test image and corresponding ground truth mask
test_img = X_test[test_img_number]
ground_truth = y_test[test_img_number]

# Visualize the original image, ground truth mask, thresholded prediction,
# and morphologically processed prediction side-by-side
visualize_results(test_img[:, :, 0], ground_truth[:, :, 0],
                  y_pred_thresholded[test_img_number], y_pred_morphed)